In [15]:
import tensorflow as  tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Flatten, Conv2D, MaxPooling2D,GlobalMaxPooling2D
from tensorflow.keras.utils import to_categorical

In [36]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)



In [37]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')

# LSTM

In [59]:
def createLSTM():
    model = Sequential()
    model.add(LSTM(128, input_shape=(28,28), activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(10, activation='softmax'))
    return model

In [60]:
model = createLSTM()
opt = tf.keras.optimizers.Adam(lr=1e-3, decay=1e-5)
model.compile(loss='categorical_crossentropy', optimizer=opt,
             metrics=['accuracy'])

model.fit(x_train.flatten(), y_train, epochs=3, batch_size=32, validation_data=(x_test, y_test))


ValueError: Error when checking input: expected lstm_7_input to have 3 dimensions, but got array with shape (47040000, 1)

In [ ]:
#adding channel dimension in order to do spatial 
x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

# CNN

In [34]:
def createCNN():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding= "Same", activation='relu',input_shape =(28,28,1)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(Dense(10, activation='softmax'))
    return model
    
    

#     model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
#                      activation ='relu', input_shape = (28,28,1)))
#     model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
#                      activation ='relu'))
#     model.add(MaxPool2D(pool_size=(2,2)))
#     model.add(Dropout(0.25))


#     model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
#                      activation ='relu'))
#     model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
#                      activation ='relu'))
#     model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
#     model.add(Dropout(0.25))


#     model.add(Flatten())
#     model.add(Dense(256, activation = "relu"))
#     model.add(Dropout(0.5))
#     model.add(Dense(10, activation = "softmax"))
#     return model

In [35]:
model = createCNN()
opt = tf.keras.optimizers.Adam(lr=1e-3, decay=1e-5)
model.compile(loss='categorical_crossentropy', optimizer=opt,
             metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=3, batch_size=32, validation_data=(x_test, y_test))


ValueError: Error when checking input: expected conv2d_11_input to have 4 dimensions, but got array with shape (60000, 28, 28)

In [21]:
x_train.shape[1:]

(28, 28)